In [17]:
import pandas as pd
import numpy as np
import os
import base64
import requests

In [18]:
API_KEY = os.getenv('TULIP_TABLE_API_KEY')
API_SECRET = os.getenv('TULIP_TABLE_API_SECRET')

In [19]:
API_CREDENTIALS = 'Basic '+base64.b64encode((API_KEY+":"+API_SECRET).encode('ascii')).decode('ascii')

In [23]:
base_url = 'https://william.tulip.co/api/v3/tables/'
table = 'fXsbaj23HEPFRnTyZ/'
SQL = 'records?limit=100&includeTotalCount=true&filterAggregator=all'
query = base_url + table + query
header = {"Authorization" : API_CREDENTIALS}
res = requests.get(query, headers=header)
print(res)
print(len(res.json()))

<Response [200]>
6


In [24]:
df = pd.json_normalize(res.json())
df.head()

,_createdAt,_sequenceNumber,_updatedAt,id,ikkuy_location,jlplh_reason,kcldd_date_downtime_start,tsclt_description,vkves_downtime,wrwyx_datetime_downtime_start
0,2023-05-27T17:55:57Z,1,2023-05-27T17:55:58Z,Machine Shop-1685210157,Machine Shop,Minor stop,2023-05-26T23:00:00-05:00,Bsbs,45441,2023-05-27T00:18:36-05:00
1,2023-05-27T17:57:32Z,2,2023-05-27T17:57:32Z,Machine Shop-1685210251,Machine Shop,Planned downtime,2023-05-26T23:00:00-05:00,Beb\n\n,88,2023-05-27T12:56:04-05:00
2,2023-05-27T18:02:09Z,3,2023-05-27T18:02:09Z,Machine Shop-1685210529,Machine Shop,Machine breakdown,2023-05-26T23:00:00-05:00,Tooling damaged. Critical. \nneed Katey’s pers...,84,2023-05-27T13:00:45-05:00
3,2023-05-27T18:18:38Z,4,2023-05-27T18:18:39Z,Assembly Line-1685211517,Assembly Line,Planned downtime,2023-05-26T23:00:00-05:00,Dd,46736,2023-05-27T00:19:42-05:00
4,2023-05-31T18:52:40Z,5,2023-05-31T18:52:41Z,Machine Shop-1685559160,Machine Shop,Planned downtime,2023-05-26T23:00:00-05:00,Aa,347622,2023-05-27T13:18:59-05:00
